In [9]:
def calcf2(P,T,rows,samples):  
    rows=rows
    samples=samples
    count=0

    y_pred=P
    y_test=T
    ftotal=0
    for i in range(0, samples):

        f=0

        somme=0
        Xrow = y_test[i]
        Yrow = y_pred[i]
        for n in range(0,rows):
            Xcell=Xrow[n]
            Ycell=Yrow[n]
            Error=Xcell-Ycell
            Eabs=Error**2
            somme=somme+Eabs
        step1=1+(1/rows)*somme
        step2=step1**(-0.5)
        step3=step2*100
        f=50*math.log10( step3 )
        ftotal=ftotal+f
    averagef2=ftotal/samples
    return(averagef2)

In [10]:
# Import Libraries and DataSet
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
import pylab
import math
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [11]:
#Training data
Content = loadtxt(r'.\TrainingData\CONTENT.csv', delimiter=',', dtype='float64')
COMPMax = loadtxt(r'.\TrainingData\COMPMax.csv', delimiter=',', dtype='float64')

COMPMax = np.array(COMPMax[np.newaxis].T)

#Target
DISS = loadtxt(r'.\TrainingData\DISS.csv', delimiter=',', dtype='float64')


X_train=np.hstack((Content,COMPMax))


In [40]:
#Validation data
COMPMaxVal = loadtxt(r'.\Validation\COMPMAX-validation.csv', delimiter=',', dtype='float64')
ContentVal = loadtxt(r'.\Validation\CONTENT-validation.csv', delimiter=',', dtype='float64')
ContentPredicted = loadtxt(r'.\ContentPredicted.csv', delimiter=',', dtype='float64')
#Target
DISSVal = loadtxt(r'.\Validation\DISS-validation.csv', delimiter=',', dtype='float64')

COMPMaxVal = np.array(COMPMaxVal[np.newaxis].T)
X_test=np.hstack((ContentPredicted,COMPMaxVal))


#y_train

y_train=DISS[:,0:53]

#y_test
y_test=DISSVal[:,0:53]

print(y_train.shape)

(111, 53)


In [47]:
from sklearn.metrics import make_scorer
# Custom scoring function
def custom_metric(y_true, y_pred):
    # Replace the following line with your own custom metric calculation (calcf2)
    score = calcf2(y_true, y_pred, 53, 22)
    return score

# Make scorer for the custom metric
custom_scorer = make_scorer(custom_metric, greater_is_better=True)


In [91]:
from sklearn.ensemble import RandomForestRegressor
trainings=1
DissolutionPoints=53
TestSize=37

average=0
#2391
for i in range(0,trainings):
    #print(i)
    regressor = RandomForestRegressor(n_estimators=100)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    f[i]=calcf2(y_test,y_pred,DissolutionPoints,TestSize)
    if f[i]<50:
        print(f[i])
    average=average+f[i]
key_max = max(f.keys(), key=(lambda k: f[k]))



# Perform cross-validation
cv_scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring=custom_scorer)


average_score = np.mean(cv_scores)
std_deviation = np.std(cv_scores)
confidence_interval = 1.96 * (std_deviation / np.sqrt(len(cv_scores)))

# Print results
print("Cross-validated scores:", cv_scores)
print("Average Score:", average_score)
print("Standard Deviation:", std_deviation)
print("95% Confidence Interval:", average_score - confidence_interval, "to", average_score + confidence_interval)



Cross-validated scores: [66.11573432 66.91553123 67.38443103 68.82064484 60.15870532]
Average Score: 65.87900934770245
Standard Deviation: 2.9923839448039127
95% Confidence Interval: 63.256069172881126 to 68.50194952252377
